##### Ana Daley (amd2yc@virginia.edu)
##### DS 5001 
##### 06 May 22

In [1]:
import pandas as pd
import numpy as np

In [2]:
import nltk

In [4]:
stop_en = nltk.corpus.stopwords.words('english')
stop_es = nltk.corpus.stopwords.words('spanish')

In [3]:
import spacy
nlpE = spacy.load('en_core_web_sm')
nlpS = spacy.load('es_core_news_sm')

In [4]:
import warnings
warnings.filterwarnings('ignore')

In [5]:
OHCO = ['title', 'language', 'volume', 'chap_num', 'para_num', 'sent_num', 'token_num']

In [6]:
DQ_Eng = pd.DataFrame(open('DQ-Eng.txt', encoding='utf-8').readlines(), columns=['line_str'])
DQ_Span = pd.DataFrame(open('DQ-Span.txt', encoding='utf-8').readlines(), columns=['line_str'])
EN_Eng = pd.DataFrame(open('EN-Eng.txt', encoding='utf-8').readlines(), columns=['line_str'])
EN_Span = pd.DataFrame(open('EN-Span.txt', encoding='utf-8').readlines(), columns=['line_str'])

In [74]:
ENE_Titles = r'^(THE LADY CORNELIA.|RINCONETE AND CORTADILLO:|THE LICENTIATE VIDRIERA; OR, DOCTOR GLASS-CASE.|THE DECEITFUL MARRIAGE|DIALOGUE BETWEEN SCIPIO AND BERGANZA,|THE LITTLE GIPSY GIRL.|THE GENEROUS LOVER.|THE SPANISH-ENGLISH LADY.|THE FORCE OF BLOOD.|THE JEALOUS ESTRAMADURAN.|THE ILLUSTRIOUS SCULLERY-MAID.|THE TWO DAMSELS.)$'
ENS_Titles = r'^(LA JITANILLA.|EL AMANTE LIBERAL.|RINCONETE Y CORTADILLO.|LA ESPAÑOLA INGLESA.|EL LICENCIADO VIDRIERA.|LA FUERZA DE LA SANGRE.|EL CELOSO ESTREMEÑO.|LA ILUSTRE FREGONA.|LAS DOS DONCELLAS.|LA SEÑORA CORNELIA.|EL CASAMIENTO ENGAÑOSO.|COLOQUIO QUE PASO ENTRE CIPION Y BERGANZA,|LA TIA FINGIDA.)$'

In [8]:
DQE_ChapPat = r'^\s*(CHAPTER)\s+(I|V|X|L|C)+[.]'
DQE_ParaPat = r'\n\n+'
DQE_SentPat = r'[.?!;:]+'
DQE_TokPat = r'[\s",-]+'

In [9]:
DQS_ChapPat = r'^\s*(Capítulo)\s+(I|V|X|L|C)+[.] |Capítulo Primero. |Capítulo primero.'
DQS_ParaPat = r'\n\n+'
DQS_SentPat = r'[.?!;:]+'
DQS_TokPat = r'[\s",-]+'

In [10]:
ENE_ParaPat = r'\n\n+'
ENE_SentPat = r'[.?!;:]+'
ENE_TokPat = r'[\s",-]+'

In [11]:
ENS_ParaPat = r'\n\n+'
ENS_SentPat = r'[.?!;:]+'
ENS_TokPat = r'[\s",-]+'

### Don Quijote

#### English

In [12]:
DQE_LINES = DQ_Eng
DQE_LINES.index.name = 'line_num'
DQE_LINES.line_str = DQE_LINES.line_str.str.replace(r'\n+', ' ',regex=True).str.strip()
DQE_LINES['line_str'] = DQE_LINES['line_str'].str.replace(r'(\w*)(\d*).jpg\s+[(]\d+\w*[)]',' ', regex=True).str.strip()
DQE_LINES['line_str'] = DQE_LINES['line_str'].str.replace('Full Size',' ').str.strip()

In [13]:
DQE_Begin = DQE_LINES.line_str.str.match(r'WHICH TREATS OF THE CHARACTER AND PURSUITS OF THE FAMOUS GENTLEMAN')
DQE_End = DQE_LINES.line_str.str.match(r'\*\*\*\s*END OF THE PROJECT GUTENBERG EBOOK DON QUIXOTE')

In [14]:
DQE_LINES = DQE_LINES.loc[DQE_LINES.loc[DQE_Begin].index[0]-1:DQE_LINES.loc[DQE_End].index[0]-11]

In [15]:
DQE_LINES['title'] = 'DON QUIXOTE'
DQE_LINES['language'] = 'English'

In [16]:
DQE_Vols = DQE_LINES.line_str.str.match(r'CHAPTER I.$')
DQE_LINES.loc[DQE_Vols, 'volume'] = [i+1 for i in range(DQE_LINES.loc[DQE_Vols].shape[0])]
DQE_LINES.volume = DQE_LINES.volume.ffill()

In [17]:
DQE_Chaps = DQE_LINES.line_str.str.match(DQE_ChapPat)
DQE_LINES.loc[DQE_Chaps, 'chap_num'] = [i+1 for i in range(DQE_LINES.loc[DQE_Chaps].shape[0])]
DQE_LINES.chap_num = DQE_LINES.chap_num.ffill()

In [18]:
DQE_LINES = DQE_LINES.dropna(subset=['chap_num'])
DQE_LINES = DQE_LINES.loc[~DQE_Chaps]
DQE_LINES.chap_num = DQE_LINES.chap_num.astype('int')

In [19]:
DQE_CHAPS = DQE_LINES.groupby(OHCO[:4]).line_str.apply(lambda x: '\n'.join(x)).to_frame('chap_str')
DQE_CHAPS['chap_str'] = DQE_CHAPS.chap_str.str.strip()

In [20]:
DQE_PARAS = DQE_CHAPS['chap_str'].str.split(DQE_ParaPat, expand=True).stack().to_frame('para_str').sort_index()
DQE_PARAS.index.names = OHCO[:5]
DQE_PARAS['para_str'] = DQE_PARAS['para_str'].str.replace(r'\n', '', regex=True)
DQE_PARAS['para_str'] = DQE_PARAS['para_str'].str.strip()
DQE_PARAS = DQE_PARAS[~DQE_PARAS['para_str'].str.match(r'^\s*$')]

In [21]:
DQE_SENTS = DQE_PARAS['para_str'].str.split(DQE_SentPat, expand=True).stack().to_frame('sent_str')
DQE_SENTS.index.names = OHCO[:6]
DQE_SENTS = DQE_SENTS[~DQE_SENTS['sent_str'].str.match(r'^\s*$')]
DQE_SENTS.sent_str = DQE_SENTS.sent_str.str.strip()

In [22]:
#DQE_TOK = DQE_SENTS['sent_str'].str.split(DQE_TokPat, expand=True).stack().to_frame('token_str')
#DQE_TOK.index.names = OHCO[:7]
#DQE_TOK['term_str'] = DQE_TOK.token_str.replace(r'[\W_]+','', regex=True).str.lower()

In [23]:
DQE_TOK = DQE_SENTS.sent_str.apply(lambda x: pd.Series(nlpE(x))).stack().to_frame('token_str')
DQE_TOK.index.names = OHCO[:7]
DQE_POS = DQE_TOK.token_str.apply(lambda x: pd.Series(x.tag_)).stack().to_frame('pos')

In [24]:
DQE_TOK = DQE_TOK.reset_index()
DQE_POS = DQE_POS.reset_index()
DQE_POS = DQE_POS.drop('level_7', axis=1)

In [25]:
DQE_TOK.token_str = DQE_TOK.token_str.apply(lambda x: str(x))
DQE_TOK['term_str'] = DQE_TOK.token_str.str.lower()
DQE_TOK = DQE_TOK.merge(DQE_POS)
DQE_TOK.set_index(OHCO[:7])

token_str  \
title       language volume chap_num para_num sent_num token_num              
DON QUIXOTE English  1.0    1        0        0        0              WHICH   
                                                       1             TREATS   
                                                       2                 OF   
                                                       3                THE   
                                                       4          CHARACTER   
...                                                                     ...   
                     2.0    126      24       8        44              fall   
                                                       45               for   
                                                       46              ever   
                                              9        0           Farewell   
                                              10       0                  ”   

                                                                   term_str  \
title       language volume chap_num para_num sent_num token_num              
DON QUIXOTE English  1.0    1        0        0        0              which   
                                                       1             treats   
                                                       2                 of   
                                                       3                the   
                                                       4          character   
...                                                                     ...   
                     2.0    126      24       8        44              fall   
                                                       45               for   
                                                       46              ever   
                                              9        0           farewell   
                                              10       0                  ”   

                                                                  pos  
title       language volume chap_num para_num sent_num token_num       
DON QUIXOTE English  1.0    1        0        0        0          WDT  
                                                       1          VBD  
                                                       2           IN  
                                                       3           DT  
                                                       4           NN  
...                                                               ...  
                     2.0    126      24       8        44          VB  
                                                       45          IN  
                                                       46          RB  
                                              9        0           VB  
                                              10       0           ''  

[429096 rows x 3 columns]

#### Spanish

In [26]:
DQS_LINES = DQ_Span
DQS_LINES.index.name = 'line_num'
DQS_LINES.line_str = DQS_LINES.line_str.str.replace(r'\n+', ' ',regex=True).str.strip()

In [27]:
DQS_Begin = DQS_LINES.line_str.str.match(r'Capítulo primero. ')
DQS_End = DQS_LINES.line_str.str.match(r'\*\*\*\s*END OF THE PROJECT GUTENBERG EBOOK DON QUIJOTE')

In [28]:
DQS_LINES = DQS_LINES.loc[DQS_LINES.loc[DQS_Begin].index[0]:DQS_LINES.loc[DQS_End].index[0]-5]

In [29]:
DQS_LINES['title'] = 'DON QUIJOTE'
DQS_LINES['language'] = 'Spanish'

In [30]:
DQS_Vols = DQS_LINES.line_str.str.match(r'Capítulo primero. |Capítulo Primero. ')
DQS_LINES.loc[DQS_Vols, 'volume'] = [i+1 for i in range(DQS_LINES.loc[DQS_Vols].shape[0])]
DQS_LINES.volume = DQS_LINES.volume.ffill()

In [31]:
DQS_Chaps = DQS_LINES.line_str.str.match(DQS_ChapPat)
DQS_LINES.loc[DQS_Chaps, 'chap_num'] = [i+1 for i in range(DQS_LINES.loc[DQS_Chaps].shape[0])]
DQS_LINES.chap_num = DQS_LINES.chap_num.ffill()

In [32]:
DQS_LINES = DQS_LINES.dropna(subset=['chap_num'])
DQS_LINES = DQS_LINES.loc[~DQS_Chaps]
DQS_LINES.chap_num = DQS_LINES.chap_num.astype('int')

In [33]:
DQS_CHAPS = DQS_LINES.groupby(OHCO[:4]).line_str.apply(lambda x: '\n'.join(x)).to_frame('chap_str')
DQS_CHAPS['chap_str'] = DQS_CHAPS.chap_str.str.strip()

In [34]:
DQS_PARAS = DQS_CHAPS['chap_str'].str.split(DQS_ParaPat, expand=True).stack().to_frame('para_str').sort_index()
DQS_PARAS.index.names = OHCO[:5]
DQS_PARAS['para_str'] = DQS_PARAS['para_str'].str.replace(r'\n', '', regex=True)
DQS_PARAS['para_str'] = DQS_PARAS['para_str'].str.strip()
DQS_PARAS = DQS_PARAS[~DQS_PARAS['para_str'].str.match(r'^\s*$')]

In [35]:
DQS_SENTS = DQS_PARAS['para_str'].str.split(DQS_SentPat, expand=True).stack().to_frame('sent_str')
DQS_SENTS.index.names = OHCO[:6]
DQS_SENTS = DQS_SENTS[~DQS_SENTS['sent_str'].str.match(r'^\s*$')]
DQS_SENTS.sent_str = DQS_SENTS.sent_str.str.strip()

In [36]:
#DQS_TOK = DQS_SENTS['sent_str'].str.split(DQS_TokPat, expand=True).stack().to_frame('token_str')
#DQS_TOK.index.names = OHCO[:7]
#DQS_TOK['term_str'] = DQS_TOK.token_str.replace(r'[\W_]+','', regex=True).str.lower()

In [37]:
DQS_TOK = DQS_SENTS.sent_str.apply(lambda x: pd.Series(nlpS(x))).stack().to_frame('token_str')
DQS_TOK.index.names = OHCO[:7]
DQS_POS = DQS_TOK.token_str.apply(lambda x: pd.Series(x.tag_)).stack().to_frame('pos')

In [38]:
DQS_TOK = DQS_TOK.reset_index()
DQS_POS = DQS_POS.reset_index()
DQS_POS = DQS_POS.drop('level_7', axis=1)

In [39]:
DQS_TOK.token_str = DQS_TOK.token_str.apply(lambda x: str(x))
DQS_TOK['term_str'] = DQS_TOK.token_str.str.lower()
DQS_TOK = DQS_TOK.merge(DQS_POS)
DQS_TOK.set_index(OHCO[:7])

token_str  \
title       language volume chap_num para_num sent_num token_num             
DON QUIJOTE Spanish  1.0    1        0        0        0               don   
                                                       1           Quijote   
                                                       2                de   
                                                       3                la   
                                                       4            Mancha   
...                                                                    ...   
                     2.0    126      37       5        80              sin   
                                                       81             duda   
                                                       82           alguna   
                                              6        0              Vale   
                                     38       0        0               Fin   

                                                                 term_str  \
title       language volume chap_num para_num sent_num token_num            
DON QUIJOTE Spanish  1.0    1        0        0        0              don   
                                                       1          quijote   
                                                       2               de   
                                                       3               la   
                                                       4           mancha   
...                                                                   ...   
                     2.0    126      37       5        80             sin   
                                                       81            duda   
                                                       82          alguna   
                                              6        0             vale   
                                     38       0        0              fin   

                                                                    pos  
title       language volume chap_num para_num sent_num token_num         
DON QUIJOTE Spanish  1.0    1        0        0        0           NOUN  
                                                       1          PROPN  
                                                       2            ADP  
                                                       3            DET  
                                                       4          PROPN  
...                                                                 ...  
                     2.0    126      37       5        80           ADP  
                                                       81          NOUN  
                                                       82           DET  
                                              6        0          PROPN  
                                     38       0        0          PROPN  

[401953 rows x 3 columns]

### Exemplary Novels

#### English

In [40]:
ENE_LINES = EN_Eng
ENE_LINES.index.name = 'line_num'
ENE_LINES.line_str = ENE_LINES.line_str.str.replace(r'\n+', ' ',regex=True).str.strip()

In [41]:
ENE_Begin = ENE_LINES.line_str.str.match(r'THE LADY CORNELIA.$')
ENE_End = ENE_LINES.line_str.str.match(r'\*\*\*\s*END OF THIS PROJECT GUTENBERG EBOOK THE EXEMPLARY NOVELS OF CERVANTES')

In [42]:
ENE_LINES = ENE_LINES.loc[ENE_LINES.loc[ENE_Begin].index[0]:ENE_LINES.loc[ENE_End].index[0]-14]

In [43]:
ENE_Stories = ENE_LINES.line_str.str.match(ENE_Titles)
ENE_LINES.loc[ENE_Stories, 'title'] = ENE_LINES.loc[ENE_Stories].line_str
ENE_LINES.title = ENE_LINES.title.ffill()

In [44]:
ENE_LINES['language'] = 'English'
ENE_LINES['volume'] = 1
ENE_LINES['chap_num'] = 1

In [45]:
ENE_CHAPS = ENE_LINES.groupby(OHCO[:4]).line_str.apply(lambda x: '\n'.join(x)).to_frame('chap_str')
ENE_CHAPS['chap_str'] = ENE_CHAPS.chap_str.str.strip()

In [46]:
ENE_PARAS = ENE_CHAPS['chap_str'].str.split(ENE_ParaPat, expand=True).stack().to_frame('para_str').sort_index()
ENE_PARAS.index.names = OHCO[:5]
ENE_PARAS['para_str'] = ENE_PARAS['para_str'].str.replace(r'\n', '', regex=True)
ENE_PARAS['para_str'] = ENE_PARAS['para_str'].str.strip()
ENE_PARAS = ENE_PARAS[~ENE_PARAS['para_str'].str.match(r'^\s*$')]

In [47]:
ENE_SENTS = ENE_PARAS['para_str'].str.split(ENE_SentPat, expand=True).stack().to_frame('sent_str')
ENE_SENTS.index.names = OHCO[:6]
ENE_SENTS = ENE_SENTS[~ENE_SENTS['sent_str'].str.match(r'^\s*$')]
ENE_SENTS.sent_str = ENE_SENTS.sent_str.str.strip()

In [48]:
ENE_TOK = ENE_SENTS.sent_str.apply(lambda x: pd.Series(nlpE(x))).stack().to_frame('token_str')
ENE_TOK.index.names = OHCO[:7]
ENE_POS = ENE_TOK.token_str.apply(lambda x: pd.Series(x.tag_)).stack().to_frame('pos')

In [49]:
ENE_TOK = ENE_TOK.reset_index()
ENE_POS = ENE_POS.reset_index()
ENE_POS = ENE_POS.drop('level_7', axis=1)

In [50]:
ENE_TOK.token_str = ENE_TOK.token_str.apply(lambda x: str(x))
ENE_TOK['term_str'] = ENE_TOK.token_str.str.lower()
ENE_TOK = ENE_TOK.merge(ENE_POS)
ENE_TOK.set_index(OHCO[:7])

token_str  \
title                                 language volume chap_num para_num sent_num token_num             
DIALOGUE BETWEEN SCIPIO AND BERGANZA, English  1      1        0        0        0          DIALOGUE   
                                                                                 1           BETWEEN   
                                                                                 2            SCIPIO   
                                                                                 3               AND   
                                                                                 4          BERGANZA   
...                                                                                              ...   
THE TWO DAMSELS.                      English  1      1        81       5        30         heroines   
                                                                                 31               of   
                                                                                 32             this   
                                                                                 33          strange   
                                                                                 34            story   

                                                                                            term_str  \
title                                 language volume chap_num para_num sent_num token_num             
DIALOGUE BETWEEN SCIPIO AND BERGANZA, English  1      1        0        0        0          dialogue   
                                                                                 1           between   
                                                                                 2            scipio   
                                                                                 3               and   
                                                                                 4          berganza   
...                                                                                              ...   
THE TWO DAMSELS.                      English  1      1        81       5        30         heroines   
                                                                                 31               of   
                                                                                 32             this   
                                                                                 33          strange   
                                                                                 34            story   

                                                                                            pos  
title                                 language volume chap_num para_num sent_num token_num       
DIALOGUE BETWEEN SCIPIO AND BERGANZA, English  1      1        0        0        0           NN  
                                                                                 1           IN  
                                                                                 2          NNS  
                                                                                 3           CC  
                                                                                 4           NN  
...                                                                                         ...  
THE TWO DAMSELS.                      English  1      1        81       5        30         NNS  
                                                                                 31          IN  
                                                                                 32          DT  
                                                                                 33          JJ  
                                                                                 34          NN  

[194683 rows x 3 columns]

#### Spanish

In [75]:
ENS_LINES = EN_Span
ENS_LINES.index.name = 'line_num'
ENS_LINES.line_str = ENS_LINES.line_str.str.replace(r'\n+', ' ',regex=True).str.strip()

In [76]:
ENS_Begin = ENS_LINES.line_str.str.match(r'LA JITANILLA.$')
ENS_End = ENS_LINES.line_str.str.match(r'\*\*\*\s*END OF THIS PROJECT GUTENBERG EBOOK ')

In [77]:
ENS_LINES = ENS_LINES.loc[ENS_LINES.loc[ENS_Begin].index[0]:ENS_LINES.loc[ENS_End].index[0]-27]

In [78]:
ENS_Stories = ENS_LINES.line_str.str.match(ENS_Titles)
ENS_LINES.loc[ENS_Stories, 'title'] = ENS_LINES.loc[ENS_Stories].line_str.str.strip()
ENS_LINES.title = ENS_LINES.title.ffill()

In [79]:
ENS_LINES['language'] = 'Spanish'
ENS_LINES['volume'] = 1
ENS_LINES['chap_num'] = 1

In [80]:
ENS_CHAPS = ENS_LINES.groupby(OHCO[:4]).line_str.apply(lambda x: '\n'.join(x)).to_frame('chap_str')
ENS_CHAPS['chap_str'] = ENS_CHAPS.chap_str.str.strip()

In [81]:
ENS_PARAS = ENS_CHAPS['chap_str'].str.split(ENS_ParaPat, expand=True).stack().to_frame('para_str').sort_index()
ENS_PARAS.index.names = OHCO[:5]
ENS_PARAS['para_str'] = ENS_PARAS['para_str'].str.replace(r'\n', '', regex=True)
ENS_PARAS['para_str'] = ENS_PARAS['para_str'].str.strip()
ENS_PARAS = ENS_PARAS[~ENS_PARAS['para_str'].str.match(r'^\s*$')]

In [82]:
ENS_SENTS = ENS_PARAS['para_str'].str.split(ENS_SentPat, expand=True).stack().to_frame('sent_str')
ENS_SENTS.index.names = OHCO[:6]
ENS_SENTS = ENS_SENTS[~ENS_SENTS['sent_str'].str.match(r'^\s*$')]
ENS_SENTS.sent_str = ENS_SENTS.sent_str.str.strip()

In [83]:
#ENS_TOK = ENS_SENTS['sent_str'].str.split(ENS_TokPat, expand=True).stack().to_frame('token_str')
#ENS_TOK.index.names = OHCO[:7]
#ENS_TOK['term_str'] = ENS_TOK.token_str.replace(r'[\W_]+','', regex=True).str.lower()

In [84]:
ENS_TOK = ENS_SENTS.sent_str.apply(lambda x: pd.Series(nlpS(x))).stack().to_frame('token_str')
ENS_TOK.index.names = OHCO[:7]
ENS_POS = ENS_TOK.token_str.apply(lambda x: pd.Series(x.tag_)).stack().to_frame('pos')

In [85]:
ENS_TOK = ENS_TOK.reset_index()
ENS_POS = ENS_POS.reset_index()
ENS_POS = ENS_POS.drop('level_7', axis=1)

In [86]:
ENS_TOK.token_str = ENS_TOK.token_str.apply(lambda x: str(x))
ENS_TOK['term_str'] = ENS_TOK.token_str.str.lower()
ENS_TOK = ENS_TOK.merge(ENS_POS)
ENS_TOK.set_index(OHCO[:7])

token_str  \
title                                      language volume chap_num para_num sent_num token_num             
COLOQUIO QUE PASO ENTRE CIPION Y BERGANZA, Spanish  1      1        0        0        0          COLOQUIO   
                                                                                      1               QUE   
                                                                                      2              PASO   
                                                                                      3             ENTRE   
                                                                                      4            CIPION   
...                                                                                                   ...   
RINCONETE Y CORTADILLO.                    Spanish  1      1        280      11       75                á   
                                                                                      76              los   
                                                                                      77              que   
                                                                                      78              los   
                                                                                      79          leyeren   

                                                                                                 term_str  \
title                                      language volume chap_num para_num sent_num token_num             
COLOQUIO QUE PASO ENTRE CIPION Y BERGANZA, Spanish  1      1        0        0        0          coloquio   
                                                                                      1               que   
                                                                                      2              paso   
                                                                                      3             entre   
                                                                                      4            cipion   
...                                                                                                   ...   
RINCONETE Y CORTADILLO.                    Spanish  1      1        280      11       75                á   
                                                                                      76              los   
                                                                                      77              que   
                                                                                      78              los   
                                                                                      79          leyeren   

                                                                                                   pos  
title                                      language volume chap_num para_num sent_num token_num         
COLOQUIO QUE PASO ENTRE CIPION Y BERGANZA, Spanish  1      1        0        0        0           VERB  
                                                                                      1          SCONJ  
                                                                                      2          PROPN  
                                                                                      3            ADP  
                                                                                      4          PROPN  
...                                                                                                ...  
RINCONETE Y CORTADILLO.                    Spanish  1      1        280      11       75         CCONJ  
                                                                                      76           DET  
                                                                                      77          PRON  
                                                                                      78          PRON  


In [87]:
ENS_TOK.title.unique()

array(['COLOQUIO QUE PASO ENTRE CIPION Y BERGANZA,', 'EL AMANTE LIBERAL.',
       'EL CASAMIENTO ENGAÑOSO.', 'EL CELOSO ESTREMEÑO.',
       'EL LICENCIADO VIDRIERA.', 'LA ESPAÑOLA INGLESA.',
       'LA FUERZA DE LA SANGRE.', 'LA ILUSTRE FREGONA.', 'LA JITANILLA.',
       'LA SEÑORA CORNELIA.', 'LA TIA FINGIDA.', 'LAS DOS DONCELLAS.',
       'RINCONETE Y CORTADILLO.'], dtype=object)

## Corpus to File

In [96]:
DQE_TOK.to_csv('DQE.csv', index=False)

In [97]:
DQS_TOK.to_csv('DQS.csv', index=False)

In [98]:
ENE_TOK.to_csv('ENE.csv', index=False)

In [99]:
ENS_TOK.to_csv('ENS.csv', index=False)

## Corpus Formatting

In [7]:
CorDQE = pd.read_csv('DQE.csv')
CorDQS = pd.read_csv('DQS.csv')
CorENE = pd.read_csv('ENE.csv')
CorENS = pd.read_csv('ENS.csv')

In [8]:
CorDQE.drop(columns=['language', 'volume'], inplace=True)
CorDQS.drop(columns=['language', 'volume'], inplace=True)
CorENE.drop(columns=['language', 'volume'], inplace=True)
CorENS.drop(columns=['language', 'volume'], inplace=True)

In [9]:
CORPUS_English = pd.concat([CorDQE, CorENE])
CORPUS_Spanish = pd.concat([CorDQS, CorENS])

In [10]:
CORPUS_English.term_str = CORPUS_English.term_str.replace(r'[\W_]+', '', regex=True)
CORPUS_Spanish.term_str = CORPUS_Spanish.term_str.replace(r'[\W_]+', '', regex=True)

In [14]:
CORPUS_English = CORPUS_English.loc[~CORPUS_English.term_str.isin(stop_en)]
CORPUS_Spanish = CORPUS_Spanish.loc[~CORPUS_Spanish.term_str.isin(stop_es)]

In [15]:
CORPUS_English.to_csv('CORPUS_English.csv', index=False)
CORPUS_Spanish.to_csv('CORPUS_Spanish.csv', index=False)